Tools 
Tools  are the interfaces that an agent , chain , or LLM can use to interact with world. They combine a few things:
1. The name of the Tool
2. A description of what the tool is 
3. JSON schema of what the inputs to the tools are
4. the function to call
5. Wheather the result of the tool should be turned directly to the user or not

In [2]:
!pip install arxiv

In [3]:
#wrapper on the top of wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max = 200)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)

In [4]:
wiki.name#output should be wikipedia
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DHRUV\\miniconda3\\envs\\rag_system\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.djangoproject.com/en/5.2/intro/tutorial01/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\DHRUV\AppData\Local\Temp\ipykernel_25152\2907006178.py:9: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C74274AF90>, search_kwargs={})

In [6]:
# #create retriever tool in langchain
# from langchain.tools.retriever import create_retriever_tool
# retriveal_tool = create_retriever_tool(retriever, "langsmith_search",
#                       "search fot information about langsmith, for any question about langsmith you mustuse this tool")
# Correct import from langchain_core
from langchain_core.tools import create_retriever_tool

# (Assuming 'retriever' is a variable you have already defined
#  e.g., as_retriever() from a vector store)
retrieval_tool = create_retriever_tool(
    retriever, 
    "langsmith_search",
    "search for information about langsmith, for any question about langsmith you must use this tool"
)

In [7]:
#arxiv tool 
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

# Step 1: Initialize the API wrapper with your config
arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)

# Step 2: Initialize the tool, passing the wrapper to it
arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_wrapper)

# Now this will work
print(arxiv_tool.name) 
# print(arxiv_tool.description)

arxiv


In [8]:
tools = [wiki, arxiv_tool, retrieval_tool]

In [9]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
import os
# This will stop the NoneType error
os.environ['GEMINI_API_KEY'] = os.getenv("GEMINI_API_KEY")


llm = ChatGoogleGenerativeAI(model = "gemini 2.5 flash" , temperature = 0.5)


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [11]:
#creation of prompt
from langchain import hub
#get the prompt to use you van modify the prompt
prompt = hub.pull("hwchase17/openai-functions-agents")
prompt.messages

ImportError: cannot import name 'hub' from 'langchain' (c:\Users\DHRUV\miniconda3\envs\rag_system\Lib\site-packages\langchain\__init__.py)

In [ ]:
#design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
                                          Answer the following questions based on only on the provided context.
                                          Think step by step before provideing a detaileed answer 
                                          your answer matter the company a lot as you are an impoartant assest to the organisation 
                                          and the organisation will tip you with $10000 if user finds your answer helpful 
                                          also great rewards awaits you if the user finds your answer helpful.
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")
# thsi  meathod will not only find the things taht are similiar in the query and the documensts that we provided 
#but rather understand the context of teh query and return teh answer based on the query by understandung the documenst

In [12]:
### Agents: so basically here what the agent is going to do is to search for the query in wiki if not found  then 
# try to search in the arxiv if still not found in the arxiv tool then it will be going to search for the query in the
# in the custom retrtieval thool that we made to be specific in all the tools that are being mentioned in the tools list 
from langchain.agents import create create_openai_tools_agent
agent = cretae_openai_tools_agent(llm, tools, prompt)

SyntaxError: invalid syntax (3939895341.py, line 4)

In [13]:
## agent executer 
from tabnanny import verbose
from langchain.agents import AgentExecuter
agent_executer = AgentExecuter(agent = agent , tools = tools , verbose = (1===1))
agent_executer

SyntaxError: invalid syntax (1735056711.py, line 4)

In [14]:
agent_executer.invoke("input": "Tell me about langsmith")

SyntaxError: invalid syntax (637821686.py, line 1)